In [ ]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 30.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux

# Ejercicio 2 - Detección de sentimiento de odio en contenido


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio se va a entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso nos vamos a centrar en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.



## 0. Imports y funciones


In [ ]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW,
   TrainingArguments,
   Trainer

)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import Dataset
import accelerate
import numpy as np
import evaluate
from sklearn.metrics import classification_report, confusion_matrix
# Balanceo de clases
from imblearn.over_sampling import SMOTE

### FUNCIONES ###

### Funcion para preparar datos ###

def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

  ### Definicion de datos para Hugginface

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que se peuda realizar.

In [ ]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [ ]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preparar datos

A la hora de preparar los datos el primer paso previo al entrenamiento, será convertir al formato aceptado para trabajar con huggingface, el formato dataset. Lo haremos al dividir el dataset en text y etiquetas.

In [ ]:
texts = dataset.tweet_text.values  # an array of strings  df.tweet_text.values
labels = dataset.cyberbullying.values  # an array of integers  df.cyberbullying.values

Dividimos el conjunto de datos en Train-Validation, y a su vez balanceamos las categorias de datos haciendo uso del objeto SMOTE de imblearn.

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size=.25,
                                                                      random_state=0,
                                                                      stratify = labels)


# En esta segunda parte, vamos a usar un conjunto de datos, dentro de train_texts y train_labels,
# con esto queremos ir validando que el proceso que estamos siguiendo, lo estamos realizando correctamente.

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts,
                                                                    train_labels,
                                                                    test_size=.2,
                                                                    random_state=0,
                                                                    stratify = train_labels)

Hago una busqueda en hagging-face para seleccionar que modelo se ajusta mejor a la naturaleza de los datos del ejercicio. En este caso usamos texto de twitter, por lo que me decido por probar el modelo **cardiffnlp/twitter-roberta-base-2019-90m**, entrenado con 90M de tweets.

https://huggingface.co/cardiffnlp/twitter-roberta-base-2019-90m


In [ ]:
model_name ="cardiffnlp/twitter-roberta-base-2019-90m"

Una vez elegido el modelo, cargamos el tokenizador.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name) # AutoTokenizer y from_pretrained

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Adicionalmente, para trabajar con el ecosistema de HuggingFace es necesario estructurar los datos que cargamos siguiendo la estructura de la librería Dataset. Ajusto a su vez que el numero maximo de tokens del vector sean 128.

In [ ]:
max_length = 128  # Puedes ajustar esto según tus necesidades

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

Antes de empezar a preparar la entrada, primero debemos establecer dos cosas: la longitud máxima de la secuencia, que es el número máximo de palabras en un documento que vamos a considerar, y el tamaño del grupo de datos (batch size). En Colab, hay restricciones que limitan el batch size a 8 y la longitud máxima de la secuencia a 96 tokens. Si quisiéramos aumentar la longitud máxima de la secuencia, podríamos reducir el batch size, pero en este caso no nos importa hacerlo.

In [ ]:
max_seq_length = 96
train_batch_size =  8
eval_batch_size = 8
test_batch_size = 8

## 4. Entrenamiento del modelo



Para empezar a entrenar, es necesario traer el modelo al entorno de ejecución. Utilizaremos la clase AutoModelForSequenceClassification(), que está diseñada para cargar modelos destinados a clasificar secuencias de texto.

Dentro de esta clase, especificaremos el número de etiquetas que tenemos en nuestro problema de clasificación, en este caso, son 2. También implementaremos unos diccionarios para mejorar la gestión y comprensión de la salida del modelo.

In [ ]:
id2label = {0: "NO_CYBERBUYLLING", 1: "CYBERBUYLLING"}
label2id = {"NO_CYBERBUYLLING": 0, "CYBERBUYLLING": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2019-90m and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definimos parámetros necesarios para el entrenamiento: Aumento el valor de per_device_train_batch_size y per_device_eval_batch_size a 16, ya que he comprobado que el proceso es más rápido.

También defino que se entrene solo una época por la necesidad de tiempo y recursos a la hora de entrenar el modelo, aunque esto implica unos resultado algo peores. Además generamos funciones de cálculo de métricas de evaluación durante el entrenamiento.

In [ ]:
training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

Creamos el objeto Trainer(), que nos permitirá ajustar el modelo:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.248100,0.231853,{'accuracy': 0.8968409281520827},{'f1': 0.940029254022428}


Trainer is attempting to log a value of "{'accuracy': 0.8968409281520827}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.940029254022428}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=1789, training_loss=0.26267325591614554, metrics={'train_runtime': 18321.9268, 'train_samples_per_second': 1.562, 'train_steps_per_second': 0.098, 'total_flos': 1882230712281600.0, 'train_loss': 0.26267325591614554, 'epoch': 1.0})

## 5. Evaluación

Para evaluar el modelo, queremos obtener las métricas de accuracy y f1 que definimos previamente.
Además vamos a hacer un classification report para comparar los resultados de este modelo con el del ejercicio 1.

In [ ]:
trainer.evaluate(test_dataset) # evaluar con test_dataset
predictions = trainer.predict(test_dataset) # Hacer predicciones en los datos de text
y_pred = predictions.predictions.argmax(axis=1) # Meter las predicciones en una lista

Trainer is attempting to log a value of "{'accuracy': 0.8962509435544745}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9397202865357439}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Extraemos las etiquetas verdaderas y calculamos el classification report:

In [ ]:
# Extraer las etiquetas de la lista

y_true = [x["labels"].item() for x in test_dataset]

#Calcular el clasification report

print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[1044  942]
 [ 295 9642]]
              precision    recall  f1-score   support

           0       0.78      0.53      0.63      1986
           1       0.91      0.97      0.94      9937

    accuracy                           0.90     11923
   macro avg       0.85      0.75      0.78     11923
weighted avg       0.89      0.90      0.89     11923



Una vez obtenido resultados, voy a comparar con los obtenidos en el ejercicio 1:

In [ ]:
RandomForest sin class_weight

[1112  495]
[ 833 8687]

              precision    recall  f1-score   support

         0.0       0.57      0.72      0.64      1607
         1.0       0.95      0.91      0.93      9520

    accuracy                           0.88     11127
   macro avg       0.76      0.81      0.78     11127
weighted avg       0.90      0.88      0.89     11127

- **Precisión**:

La accuracy o precisión, representa el porcentaje total de valores correctamente clasificados, tanto positivos como negativos, es una buena métrica si las categorías están equilibradas, pero algo menos util si hay desbalanceo de datos.

El modelo RandomForest, tiene una precisión de tan solo 0.57 frente a los 0.78 del modelo BERT para la clase minoritaria 0, sin embargo obtiene un mayor rendimiento para la clase mayoritaria 1, 0.95 frente a 0.91

El modelo basado en BERT tiene una precisión general más alta (macro avg = 0.85) en comparación con el modelo RandomForest (macro avg = 0.76), sin embargo,  en el calculo del promedio ponderado teniendo en cuenta el peso  de cada clase (weighted avg), el modelo RandomForest es algo mejor que el modelo BERT (0.90 vs 0.89)

El modelo basado en BERT tiene una precisión general más alta, pero el modelo RandomForest supera ligeramente al modelo BERT cuando se considera el desbalance de clases en el conjunto de datos.

- **Recall**:

El recall por definición es el % de casos identificados correctamente como positivos del total de positivos verdaderos. Nos da información sobre el rendimiento del clasificador.

En el modelo BERT, tenemos un recall de 0.53 en la clase minoritaria y de 0.97 en la clase mayoritaria, frente a unos valores de 0.72 en la clase 0 y 0.91 en la clase 1 para el modelo de randomForest, en general el modelo BERT, tiene un valor de 0.75 en la macro avg frente a 0.81 por parte del modelo RandomForest del ejercicio1, sin embargo, en cuanto al weighted avg el modelo BERT es mejor que el modelo RandonForest.

Respecto a la metrica recall,aunque el modelo basado en BERT tiene un recall más bajo en la clase minoritaria y un valor más bajo de macro avg en comparación con el modelo RandomForest, su mejor rendimiento en el weighted avg indica que puede ser más adecuado para conjuntos de datos desequilibrados. Pero es muy considerable el valor tan bajo que obtiene en cuanto a la clase minoritaria ( solo 0.53), BERT falla mucho a la hora de identificar positivos de la clase minoritaria.

- **f1 score:**

El F1-score es una medida que combina la precisión y el recall en una sola métrica.

Para la clase 0, el F1-score del modelo RandomForest es de 0.64, ligeramente más alto que el del modelo BERT (0.64 vs 0.63). Mientras que para la clase 1 el valor es de 0.93, ligeramente más bajo que el del modelo BERT (0.93 vs 0.94).

La precisión general del modelo RandomForest es del 88%, ligeramente más baja que la del modelo BERT (88% vs 90%), y los valores de macro avg y weighted avg para F1-score son similares para ambos modelos (0.78 y 0.89, respectivamente).

**Conclusiones**

Modelo BERT:

El modelo Bert elegido, **"cardiffnlp/twitter-roberta-base-2019-90m"** https://huggingface.co/cardiffnlp/twitter-roberta-base-2019-90m es un modelo predefinido que ha sido entrenado en grandes corpus de tweets (90M) para aprender representaciones de palabras contextualizadas en base a un conjunto de tweets masivo. No haber realizado el entrenamiento durante varias épocas hace que sea posible que no se estén aprovechando al máximo las capacidades del modelo. Esto se ha visto reflejado en el classfication report, que aun obteniendo unas buenas puntuaciones en general, pueden mejorar bastante.

Modelo RandomForest:

El modelo propio RandomForest, por otro lado, es un modelo de aprendizaje automático que ha sido procesado, configurado y entrenado de forma personalizada. Esto proporciona un mayor control sobre su comportamiento y rendimiento, permitiendo ajustar hiperparámetros, seleccionar características relevantes y aplicar técnicas de preprocesamiento de datos específicas para optimizar su desempeño en la tarea de clasificación. Al ser un modelo más flexible y adaptable, RandomForest puede ser más fácil de ajustar y afinar para satisfacer las necesidades específicas del problema, como podría ser tener capacidad de manejar diferentes tipos de datos y distribuciones de clases.

En resumen, ambos modelos tienen fortalezas y debilidades. El modelo basado en BERT muestra un rendimiento más alto en términos de precisión general y F1-score para la clase 1, mientras que el modelo RandomForest muestra un mejor balance entre precisión y recall.